In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ecdsa
import math

# Plot params
plt.rcParams["figure.figsize"] = [7.50, 7.50]
plt.rcParams["figure.autolayout"] = True

CURVE = ecdsa.SECP256k1

y, x = np.ogrid[-25:25:100j, -2:7:100j]
plt.contour(
    x.ravel(),
    y.ravel(),
    pow(y, 2) - pow(x, 3) - x * CURVE.curve.a() - CURVE.curve.b(),
    [0],
)
plt.grid()
plt.show()

# Modulo Arithmetic

- Remember that (-a)^(-1) == -(a)^(-1)
(-a)^(-1) = (-1 * a)^(-1) = (-1)^(-1) * (a)^(-1) = -1 * (a)^(-1) = -(a)^(-1)
-1's inverse is always -1 : -1 * -1 = 1


# ECDSA: Basics

```
    pubkey = privatekey * G
    R = (x_r, y_r) =  k * G
    r = x_r
    s = k^-1 * (h + d r)
```

## Signature verification

G is the generator
d the private key
p the public key

R = k * G (def)
  = s^(-1) * G * (h + r * d)  (def of s)
  = s^(-1) * G * h +  s^(-1) * G * r * d
  = s^(-1) * G * h +  s^(-1) * r * p (def of p)

We can compute R' = s^(-1) * G * h +  s^(-1) * r * p. If R' and R have the same x coordinate, the signature is valid.


## Alternative s value
If (r, s) is a valid signature, then (r, order - s) is also a valid signature.

Preliminaries:
  If P (xp, yp) is a point on the curve and n an integer, then we have:

  n * P = (x, y1)
  n * -P = (x, -y1)

  The explanation is trivial: -P = (xp, -yp) by definition. As the curve is symmetric over the x axis, it is obvious that n * P and n * (-P) share the same x coordinate and that those two points are symmetric with respect to the x axis.

Proof:
  s' = order - s = -s (we are in modular arithmetic)

  R = h * s^(-1) * G + r * s(-1) * pk
  <=> R * s = h * G + r * pk

  R' = h * s'^(-1) * G + r * s'^(-1) * pk
  <=> R' * s' = h * G + r * pk

  Thus R * s = R' * s', 
  <=> R * s = R' * -s
  <=> R = - R'

As the verification only checks the x coordinate, both signatures are still valid.


In [ ]:
from ecdsa.keys import SigningKey
from ecdsa.util import sigdecode_string, sigencode_string

nonce = 100
data = b"01"
sk = SigningKey.from_secret_exponent(12, curve=CURVE)

# Sign a first time
sig = sk.sign_digest(data, k=nonce)
r, s = sigdecode_string(sig, CURVE.order)

# Generate alternative sig
alt_sig = sigencode_string(r, CURVE.order - s, CURVE.order)

assert sig != alt_sig

vk = sk.get_verifying_key()
vk.verify_digest(
    sig,
    data,
)
vk.verify_digest(
    alt_sig,
    data,
)

## Two nonces can generate the same r
For the same x coordinate, there are always two corresponding points, i.e. 2 corresponding y coordinates.
Thus, every r can be generated from 2 distinct nonces leading to two different points.

In [ ]:
order = ecdsa.SECP256k1.order

nonce1 = 57896044618658097711785492504343953926418782139537452191302581570759080747169
nonce2 = 57896044618658097711785492504343953926418782139537452191302581570759080747168

pt1 = nonce1 * ecdsa.SECP256k1.generator
pt2 = nonce2 * ecdsa.SECP256k1.generator

print("Pt1 :", pt1.x(), pt1.y())
print("Pt2 :", pt2.x(), pt2.y())

# Nonce recovery from repeated nonces

If the same nonce is used for 2 distinct signatures, then it is possible to retrieve the nonce.

- r = k * G
- s1 = k ^(-1)(h1 + d r)
- s2  = k ^(-1)(h2 + d r)

-> k = (h1 - h2)(s1 - s2)^(-1)


In [ ]:
from ecdsa.keys import SigningKey
from ecdsa.util import sigdecode_string, sigencode_string

k = 100
d = 13

sk = SigningKey.from_secret_exponent(d, curve=CURVE)

# First signature
h1 = "01"
sig1 = sk.sign_digest(bytes.fromhex(h1), k=k)
r, s1 = sigdecode_string(sig1, CURVE.order)

# Second signature
h2 = "02"
sig2 = sk.sign_digest(bytes.fromhex(h2), k=k)
_, s2 = sigdecode_string(sig2, CURVE.order)

## Recovering nonce: Warning
However, one must be careful that the signer might have published s1 and s2 and might have instead published -s1 or/and -s2.


s1 s2: successful recovery of k
-s1 s2: error
s1 -s2: error
-s1 -s2: successful recovery of -k   /!\

Cases: -s1 s2 and s1 s2
________________________
Easy to spot as the recovered nonce k' won't be such that the x coordinate of  (k' * G) == r.

Cases: -s1 -s2
________________________
What we recover is:  k' = (h1 - h2) ((-s1) - (-s2))^(-1) = (h1 - h2) (- (s1 - s2))^(-1) = - (h1 - h2) (s1 - s2)^(-1) = -k
In that case, one must must add an extra verification layer to ensure that the recovered nonce is correct.

In [ ]:
s1_bis, s2_bis = CURVE.order - s1, CURVE.order - s2

recovered_nonce = (
    (int(h1, 16) - int(h2, 16)) * pow(s1_bis - s2_bis, -1, CURVE.order) % CURVE.order
)
recovered_R = recovered_nonce * CURVE.generator
# Insufficient checking
assert recovered_R.x() == r

# The private key is not retrieved correctly
priv_key = pow(r, -1, CURVE.order) * (s2 * recovered_nonce - int(h2, 16)) % CURVE.order
print(priv_key == sk.privkey.secret_multiplier)

In [ ]:
from ecdsa.util import sigdecode_string
from random import randbytes, randint


def negated_nonce(curve=ecdsa.SECP256k1):
    d1 = 111
    sk1 = ecdsa.SigningKey.from_secret_exponent(secexp=d1, curve=curve)
    h = randbytes(randint(1, 150))

    k1 = 100
    sig1 = sk1.sign(h, k=k1)
    r, s = sigdecode_string(sig1, curve.order)

    # First, it is obvious that -k1 also generate the same signature as it will produce an intermediary point R that shares the same x coordinate
    sig2 = sk1.sign(h, k=(-k1 % curve.order))
    r2, s2 = sigdecode_string(sig1, curve.order)
    # The hex encoding of those signatures are different, but
    assert sig1 != sig2
    # their components r and s are the same modulo curve.order
    # In fact, s2 = -s
    assert r == r2 and s == s2


negated_nonce()

In [ ]:
from ecdsa.util import sigdecode_string
from random import randbytes, randint


def mirror_key(curve=ecdsa.SECP256k1):
    """
    Given a private key 'd' and a signature '(r, s)' for a message 'h', there exists a second private key for which this signature is also valid for this same message.

    If k was used for the first private key, -k is the nonce for the second private key.
    """
    d1 = 50745265469448826586412566265716382129545677401674712567676050673179507570453
    sk1 = ecdsa.SigningKey.from_secret_exponent(secexp=d1, curve=curve)
    h = randbytes(randint(1, 150))

    k1 = 57896044618658097711785492504343953926418782139537452191302581570759080747168
    sig1 = sk1.sign(h, k=k1)
    r, s = sigdecode_string(sig1, curve.order)

    # There exist a second distinct private key that would generate the same signature
    d2 = (pow(r, -1, curve.order) * s * (-2) * k1 + d1) % curve.order
    sk2 = ecdsa.SigningKey.from_secret_exponent(secexp=d2, curve=curve)
    assert d2 != d1 and sk1 != sk2

    k2 = -k1 % order
    sig2 = sk2.sign(h, k=k2)
    assert sig1 == sig2


mirror_key()